In [ ]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings(action='ignore')
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import Dataset,DataLoader
import torch.optim as optim
torch.manual_seed(1015)
# define 'device' to upload tensor in gpu
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [ ]:
# EarlyStopping
import numpy as np
import torch

class EarlyStopping:
    """Early stops the training if validation loss doesn't improve after a given patience."""
    def __init__(self, patience=7, verbose=False, delta=0, path='checkpoint.pt', trace_func=print):
        """
        Args:
            patience (int): How long to wait after last time validation loss improved.
                            Default: 7
            verbose (bool): If True, prints a message for each validation loss improvement. 
                            Default: False
            delta (float): Minimum change in the monitored quantity to qualify as an improvement.
                            Default: 0
            path (str): Path for the checkpoint to be saved to.
                            Default: 'checkpoint.pt'
            trace_func (function): trace print function.
                            Default: print            
        """
        self.patience = patience
        self.verbose = verbose
        self.counter = 0
        self.best_score = None
        self.early_stop = False
        self.val_loss_min = np.Inf
        self.delta = delta
        self.path = path
        self.trace_func = trace_func
    def __call__(self, val_loss, model):

        score = -val_loss

        if self.best_score is None:
            self.best_score = score
            self.save_checkpoint(val_loss, model)
        elif score < self.best_score + self.delta:
            self.counter += 1
            self.trace_func(f'EarlyStopping counter: {self.counter} out of {self.patience}')
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_score = score
            self.save_checkpoint(val_loss, model)
            self.counter = 0

    def save_checkpoint(self, val_loss, model):
        '''Saves model when validation loss decrease.'''
        if self.verbose:
            self.trace_func(f'Validation loss decreased ({self.val_loss_min:.6f} --> {val_loss:.6f}).  Saving model ...')
        torch.save(model.state_dict(), self.path)
        self.val_loss_min = val_loss

In [ ]:
train = pd.read_csv("/content/drive/MyDrive/dacon/daconcup/Data/train.csv", encoding = 'euc-kr')
train['DateTime'] = pd.to_datetime(train.DateTime)
train['date'] = train.DateTime.dt.date
train  = train.groupby('date').sum().reset_index()
train['date'] = pd.to_datetime(train['date'] )

In [ ]:
# x에 is_holiday 변수 추가
import os
os.chdir('/content/drive/MyDrive/dacon/daconcup/')
import utils
train = utils.add_isHoliday_column(train)

In [ ]:
train

,date,사용자,세션,신규방문자,페이지뷰,isHoliday
0,2018-09-09,281,266,73,1826,1
1,2018-09-10,264,247,51,2092,0
2,2018-09-11,329,310,58,1998,0
3,2018-09-12,300,287,45,2595,0
4,2018-09-13,378,344,50,3845,0
...,...,...,...,...,...,...
787,2020-11-04,4516,4472,1196,112683,0
788,2020-11-05,4155,4037,1044,102901,0
789,2020-11-06,3663,3576,825,88015,0
790,2020-11-07,2472,2417,531,57386,1


In [ ]:
total_df = train.copy()


In [ ]:
# 마지막 10일을 validation set으로!

# minmax scaling
    # >> valid set 제외한 train set만 사용해서 scaling
mini = train.iloc[:-10,1:-1].min()
size = train.iloc[:-10,1:-1].max() - train.iloc[:,1:-1].min()
train.iloc[:,1:-1] = (train.iloc[:,1:-1] -  mini) / size

input_window = 30 # 30일씩 데이터를 넣음
output_window = 7 # 이후 7일을 예측

# (a, b, c) -> a: 데이터 수, b: input (이전 30일 데이터, isHoliday), c: output 수
# 사용자 수 -> 
train_window_x = np.zeros((train.shape[0] - (input_window + output_window + 10), input_window + 1, 4)) # 공휴일여부 추가
train_window_y = np.zeros((train.shape[0] - (input_window + output_window + 10), output_window, 4))
valid_window_x = np.zeros((10, input_window+1, 4))
valid_window_y = np.zeros((10, output_window, 4))

for start in range(train.shape[0] - (input_window + output_window + 10)):
    end = start + input_window    
    train_window_x[start,:-1, :] = train.iloc[start : end                , 1: -1].values # 1~30일 사용자수,...4가지
    train_window_x[start, -1, :] = train.iloc[end, -1]                                   # 31일의 isholiday 
    train_window_y[start,:, :] = train.iloc[end   : end + output_window, 1: -1].values   # 1일~7일 사용자수,....4가지

i = 0
for start in range(train.shape[0] - (input_window + output_window + 9), train.shape[0] - (input_window + output_window)):
    end = start + input_window
    valid_window_x[i, :-1, :] = train.iloc[start : end , 1:-1].values
    valid_window_x[i, -1, :] = train.iloc[end, -1]
    valid_window_y[i, :, :] = train.iloc[end : end + output_window, 1:-1].values
    i += 1

In [ ]:
print(train_window_x.shape, train_window_y.shape)
print(valid_window_x.shape, valid_window_y.shape)

(745, 31, 4) (745, 7, 4)
(10, 31, 4) (10, 7, 4)


In [ ]:
class LSTM(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(LSTM, self).__init__()
        self.hidden_size = hidden_size
        self.lstm = nn.LSTM(input_size = input_size,
                            hidden_size = hidden_size,
                            batch_first=True)
        self.hidden_lstm = nn.LSTM(input_size = hidden_size,
                                   hidden_size = hidden_size,
                                   batch_first=True)
        
        self.time_fc = nn.Linear(hidden_size, 4)
    
    def forward(self, x_time):
    
        out_time, _ = self.lstm(x_time)
        out_time, _ = self.hidden_lstm(out_time)
        out_time, _ = self.hidden_lstm(out_time)
        out_time, _ = self.hidden_lstm(out_time)
        
        
        out_time = self.time_fc(out_time[:,-7:, :])
        
        return out_time.view(-1,7,4)
    
model = LSTM(input_size = 4, hidden_size = 30).to(device)

In [ ]:
train_window_x = torch.tensor(train_window_x).float().to(device)
train_window_y = torch.tensor(train_window_y).float().to(device)

valid_window_x = torch.tensor(valid_window_x).float().to(device)
valid_window_y = torch.tensor(valid_window_y).float().to(device)

In [ ]:
# Train model
optimizer = torch.optim.Adam(model.parameters(), lr = 1e-2)
criterion = nn.MSELoss(size_average = True)
num_epochs  = 800
train_error = []
valid_error = []

# initialize the early_stopping object
early_stopping = EarlyStopping(patience=20, verbose=True)

for t in range(num_epochs):
    train_pred = model(train_window_x)
    train_loss = criterion(train_pred, train_window_y) ### trend
    train_error.append(train_loss)
    
    optimizer.zero_grad()
    train_loss.backward()
    optimizer.step()

    valid_pred = model(valid_window_x)
    valid_loss = criterion(valid_pred, valid_window_y)
    valid_error.append(valid_loss)
    

    if t % 20 == 0 and t !=0:
        print_msg = (f'[{t:>{3}}/{num_epochs}] ' +
                     f'train_loss: {train_loss.item():.5f} ' +
                     f'valid_loss: {valid_loss.item():.5f}')
        print(print_msg)

    early_stopping(valid_loss, model)
    if early_stopping.early_stop:
        print('Early_stopping')
        break

Validation loss decreased (inf --> 0.288244).  Saving model ...
Validation loss decreased (0.288244 --> 0.173801).  Saving model ...
EarlyStopping counter: 1 out of 20
EarlyStopping counter: 2 out of 20
EarlyStopping counter: 3 out of 20
EarlyStopping counter: 4 out of 20
EarlyStopping counter: 5 out of 20
EarlyStopping counter: 6 out of 20
EarlyStopping counter: 7 out of 20
EarlyStopping counter: 8 out of 20
EarlyStopping counter: 9 out of 20
Validation loss decreased (0.173801 --> 0.170608).  Saving model ...
Validation loss decreased (0.170608 --> 0.164884).  Saving model ...
Validation loss decreased (0.164884 --> 0.153610).  Saving model ...
Validation loss decreased (0.153610 --> 0.132892).  Saving model ...
Validation loss decreased (0.132892 --> 0.102624).  Saving model ...
Validation loss decreased (0.102624 --> 0.078289).  Saving model ...
Validation loss decreased (0.078289 --> 0.077476).  Saving model ...
EarlyStopping counter: 1 out of 20
EarlyStopping counter: 2 out of 20

In [ ]:
submission = pd.read_csv("/content/drive/MyDrive/dacon/daconcup/Data/submission.csv", encoding = 'euc-kr')
sub_copy = submission.copy()
sub_copy['date'] = pd.to_datetime(sub_copy['DateTime'])
sub_copy = utils.add_isHoliday_column(sub_copy)
sub_holiday_lst = sub_copy.isHoliday.tolist()

In [ ]:
submission

,DateTime,사용자,세션,신규방문자,페이지뷰
0,2020-11-09,0,0,0,0
1,2020-11-10,0,0,0,0
2,2020-11-11,0,0,0,0
3,2020-11-12,0,0,0,0
4,2020-11-13,0,0,0,0
...,...,...,...,...,...
56,2021-01-04,0,0,0,0
57,2021-01-05,0,0,0,0
58,2021-01-06,0,0,0,0
59,2021-01-07,0,0,0,0


In [ ]:
#last_month = torch.tensor(window_x[-1,:,:][np.newaxis,...]).float().to(device) <- 수정 전
last_month = np.zeros((1, 31, 4))
last_month[:, :-1, :] = train.iloc[-30:,1:-1].values[np.newaxis,...] # <- 수정 후
last_month[:, -1, :] = sub_copy.iloc[0, -1]
last_month = torch.tensor(last_month).float().to(device) # <- 수정 후

for start in range((len(submission) - output_window)//7 + 2): # 0, ..., 8
    start = start * 7
    next_week = model(last_month)
    #last_month = torch.cat([last_month[-7:], next_week], axis = 1) <- 수정 전
    last_month = torch.cat([last_month[:,7:-1,:], next_week], axis = 1) # tensor # <- 수정 후

    isholiday_arr = np.array([[[sub_holiday_lst[start]]*4]])
    isholiday = torch.tensor(isholiday_arr).float().to(device)
    last_month = torch.cat([last_month, isholiday], axis=1)

    pred_week = next_week.cpu().detach().numpy().reshape(output_window,4)
    pred_week = pred_week * size.values + mini.values # scaling 역함수
    pred_week = pred_week.astype(int)
    
    if start/7 == (len(submission) - output_window)//7 + 1:
        submission.iloc[start :, 1:] = pred_week[-submission.iloc[start :, 1:].shape[0]:,:]
    else:
        submission.iloc[start : start + output_window, 1:] = pred_week
submission

In [ ]:
submission.to_csv('/content/drive/MyDrive/dacon/daconcup/submission/02_baseline_plus_isholiday_earlystopping.csv', index = False, encoding = 'euc-kr')